In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import numpy as np
from shapely.ops import unary_union
from shapely.geometry import LineString, MultiLineString, Point
from matplotlib_scalebar.scalebar import ScaleBar

# --- 1. Load Data ---
# Ensure the file paths below are correct for your system.

# Municipalities Shapefile
shapefile_path_municipalities = r"MT_Municipios_2022\MT_Municipios_2022.shp"
municipalities = gpd.read_file(shapefile_path_municipalities)

# Highways Shapefile
shapefile_path_highways = r"Rodovias\SNV_202410A.shp"
highways = gpd.read_file(shapefile_path_highways)

# Colonization Data from Excel
file_path_excel = "Table 1.xlsx"
colonization = pd.read_excel(file_path_excel, sheet_name='Planilha1')


# --- 2. Process and Merge Data (with Reprojection) ---

# ⭐ BEST PRACTICE: Reproject data to a projected CRS with units in meters.
# This ensures accurate distance measurements for the scale bar.
# SIRGAS 2000 / UTM Zone 21S (EPSG:31981) is appropriate for Mato Grosso.
projected_crs = "EPSG:31981"
municipalities = municipalities.to_crs(projected_crs)
highways = highways.to_crs(projected_crs)

# Filter highways for Mato Grosso (MT)
filtered_highways_MT = highways[highways['sg_uf'] == 'MT']

# Group highways by their BR number and combine their geometries
def combine_geometries(group):
    return unary_union(group.geometry)

combined_geometries_series = filtered_highways_MT.groupby('vl_br').apply(combine_geometries)
combined_geometries = gpd.GeoDataFrame(geometry=combined_geometries_series.values, crs=filtered_highways_MT.crs)
combined_geometries['vl_br'] = combined_geometries_series.index

# Join municipal data with colonization data
merged = pd.merge(municipalities, colonization, left_on='NM_MUN', right_on='City', how='inner')

# Filter for cities that have a valid colonization year
colonization_cities = merged[merged['Year'].notna()].copy()


# --- 3. Function to normalize angle (keep text readable) ---
def normalize_angle(angle):
    """Normalize angle to keep text readable (not upside down)"""
    angle = angle % 360
    if angle > 90 and angle <= 270:
        angle = angle - 180
    return angle

# --- 4. Function to place markers at specific distance intervals ---
def get_distance_based_markers(highway_geom, marker_distance=50000, start_offset=10000, min_segment_length=5000):
    """
    Place markers at specific distance intervals along highway geometry.
    NOTE: Distances are now in meters due to reprojection.
    """
    positions = []

    def calculate_angle_at_distance(line, distance):
        angle_calc_distance = min(1000, line.length * 0.05)
        prev_distance = max(0, distance - angle_calc_distance)
        next_distance = min(line.length, distance + angle_calc_distance)
        try:
            p1 = line.interpolate(prev_distance)
            p2 = line.interpolate(next_distance)
            return normalize_angle(np.degrees(np.arctan2(p2.y - p1.y, p2.x - p1.x)))
        except:
            return 0

    def process_line_segment(line, start_offset, marker_distance):
        line_positions = []
        if line.length < min_segment_length:
            return line_positions
        current_distance = start_offset
        while current_distance < (line.length - start_offset):
            try:
                point = line.interpolate(current_distance)
                angle = calculate_angle_at_distance(line, current_distance)
                line_positions.append((point.x, point.y, angle))
                current_distance += marker_distance
            except Exception as e:
                current_distance += marker_distance
                continue
        return line_positions

    if isinstance(highway_geom, MultiLineString):
        lines = list(highway_geom.geoms)
        significant_lines = [line for line in lines if line.length > min_segment_length]
        significant_lines.sort(key=lambda x: x.length, reverse=True)
        for line in significant_lines:
            line_positions = process_line_segment(line, start_offset, marker_distance)
            positions.extend(line_positions)
    else:
        positions = process_line_segment(highway_geom, start_offset, marker_distance)
    return positions


# --- 5. Plotting the Map ---

fig, ax = plt.subplots(figsize=(12, 14))

# Plot the base map of all municipalities
municipalities.plot(ax=ax, color="white", edgecolor='darkgray', linewidth=0.5)


# --- 6. Plot Highways ---
combined_geometries.plot(ax=ax, color='black', linestyle='-', linewidth=1.5)


# --- 7. Add Distance-Based Highway Markers ---
highway_markers = {
    '163': 'o', '364': 's', '158': '^', '070': 'h',
    '080': 'D', '174': 'p', '242': '*', '251': 'P',
}

# DISTANCE PARAMETERS (now in meters)
MARKER_DISTANCE = 50000      # 50 km between markers
START_OFFSET = 10000         # 10 km from start/end
MIN_SEGMENT_LENGTH = 5000    # Minimum 5 km segment length

legend_elements_highways = []
unique_highways = sorted(combined_geometries['vl_br'].unique())
for br_name in unique_highways:
    highway_geom = combined_geometries[combined_geometries['vl_br'] == br_name].geometry.iloc[0]
    marker = highway_markers.get(str(br_name), 'o')
    positions = get_distance_based_markers(
        highway_geom,
        marker_distance=MARKER_DISTANCE,
        start_offset=START_OFFSET,
        min_segment_length=MIN_SEGMENT_LENGTH
    )
    for x, y, angle in positions:
        ax.scatter(x, y, marker=marker, s=70, c='black', edgecolors='white',
                   linewidth=1.2, zorder=5)
    legend_elements_highways.append(
        Line2D([0], [0], marker=marker, color='w', markerfacecolor='black',
               markeredgecolor='white', markersize=8, label=f'BR-{br_name}')
    )


# --- 8. Plot, Number, and Create Legend for Cities ---

legend_elements_colonization = []
sorted_cities = sorted(colonization_cities['NM_MUN'].unique())

for i, city_name in enumerate(sorted_cities):
    city_geo = colonization_cities[colonization_cities['NM_MUN'] == city_name]
    city_geo.plot(ax=ax, facecolor='#d3d3d3', edgecolor='black', linewidth=0.7)
    label_pos = city_geo.geometry.iloc[0].representative_point()
    ax.text(label_pos.x, label_pos.y, str(i + 1),
            ha='center', va='center', fontsize=9, weight='bold', color='black')
    legend_elements_colonization.append(
        Patch(facecolor='#d3d3d3', edgecolor='black', label=f"{i+1}: {city_name}")
    )

# Create the legend for Highways
highway_legend = ax.legend(
    handles=legend_elements_highways,
    title="Highways",
    loc='upper right',
    bbox_to_anchor=(1.0, 1.0),
    prop={'size': 8},
    fancybox=True
)

# Create the legend for Colonization Cities
city_legend = ax.legend(
    handles=legend_elements_colonization,
    title="Colonization cities",
    loc='upper center',
    bbox_to_anchor=(0.5, -0.05), # Position below the plot
    ncol=4, # Adjust columns as needed
    prop={'size': 9},
    fancybox=True
)
ax.add_artist(highway_legend)


# --- 9. Final Touches with Dual Scale Bars ---

# Create a custom kilometers scale bar to match the miles one
def add_kilometers_scale_bar(ax, location="lower right"):
    """Add a custom kilometers scale bar"""
    
    # Get the current axis limits to determine appropriate scale
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # Use 200 km as specified
    scale_length_km = 200
    scale_length_m = scale_length_km * 1000  # Convert to meters
    
    # Position the scale bar
    if location == "lower right":
        x_pos = xlim[1] - (xlim[1] - xlim[0]) * 0.25
        y_pos = ylim[0] + (ylim[1] - ylim[0]) * 0.05
    else:
        x_pos = xlim[0] + (xlim[1] - xlim[0]) * 0.05
        y_pos = ylim[0] + (ylim[1] - ylim[0]) * 0.05
    
    # Draw the scale bar line
    ax.plot([x_pos, x_pos + scale_length_m], [y_pos, y_pos], 
            color='black', linewidth=2, solid_capstyle='butt')
    
    # Add tick marks at the ends
    tick_height = (ylim[1] - ylim[0]) * 0.01
    ax.plot([x_pos, x_pos], [y_pos - tick_height/2, y_pos + tick_height/2], 
            color='black', linewidth=2)
    ax.plot([x_pos + scale_length_m, x_pos + scale_length_m], 
            [y_pos - tick_height/2, y_pos + tick_height/2], 
            color='black', linewidth=2)
    
    # Add labels (removed "km" from the main label)
    ax.text(x_pos + scale_length_m/2, y_pos + tick_height*2, 
            'Kilometers', 
            ha='center', va='bottom', fontsize=10, color='black')
    ax.text(x_pos + scale_length_m/2, y_pos + tick_height*4, 
            f'{scale_length_km}', 
            ha='center', va='bottom', fontsize=10, color='black')
    ax.text(x_pos, y_pos - tick_height*2, '0', 
            ha='center', va='top', fontsize=9, color='black')
    ax.text(x_pos + scale_length_m, y_pos - tick_height*2, 
            f'{scale_length_km}', 
            ha='center', va='top', fontsize=9, color='black')

# Add the custom kilometers scale bar
add_kilometers_scale_bar(ax, location="lower right")

# Add a custom scale bar for miles
# Since matplotlib-scalebar doesn't support miles, we'll create our own
# 1 km = 0.621371 miles
def add_miles_scale_bar(ax, location="lower left"):
    """Add a custom miles scale bar"""
    
    # Get the current axis limits to determine appropriate scale
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # Use 200 km as the base (same as kilometers scale bar)
    scale_length_km = 200
    scale_length_m = scale_length_km * 1000  # Convert to meters
    
    # Convert to miles with 2 decimal places
    scale_length_miles = scale_length_km * 0.621371
    
    # Position the scale bar
    if location == "lower left":
        x_pos = xlim[0] + (xlim[1] - xlim[0]) * 0.05
        y_pos = ylim[0] + (ylim[1] - ylim[0]) * 0.05
    elif location == "lower right":
        x_pos = xlim[1] - (xlim[1] - xlim[0]) * 0.25
        y_pos = ylim[0] + (ylim[1] - ylim[0]) * 0.05
    else:
        x_pos = xlim[0] + (xlim[1] - xlim[0]) * 0.05
        y_pos = ylim[0] + (ylim[1] - ylim[0]) * 0.05
    
    # Draw the scale bar line
    ax.plot([x_pos, x_pos + scale_length_m], [y_pos, y_pos], 
            color='black', linewidth=2, solid_capstyle='butt')
    
    # Add tick marks at the ends
    tick_height = (ylim[1] - ylim[0]) * 0.01
    ax.plot([x_pos, x_pos], [y_pos - tick_height/2, y_pos + tick_height/2], 
            color='black', linewidth=2)
    ax.plot([x_pos + scale_length_m, x_pos + scale_length_m], 
            [y_pos - tick_height/2, y_pos + tick_height/2], 
            color='black', linewidth=2)
    
    # Add labels with proper decimal formatting
    ax.text(x_pos + scale_length_m/2, y_pos + tick_height*2, 
            f'{scale_length_miles:.2f} Miles', 
            ha='center', va='bottom', fontsize=10, color='black')
    ax.text(x_pos, y_pos - tick_height*2, '0', 
            ha='center', va='top', fontsize=9, color='black')
    ax.text(x_pos + scale_length_m, y_pos - tick_height*2, 
            f'{scale_length_miles:.2f}', 
            ha='center', va='top', fontsize=9, color='black')

# Add the custom amiles scale bar
add_miles_scale_bar(ax, location="lower left")

# Clean up the plot appearance
ax.set_axis_off()
fig.subplots_adjust(bottom=0.25, right=0.85)  # Added right margin to ensure scale bar fits

# Save the figure
output_path = "Land Market and Transports.jpg"
plt.savefig(output_path, dpi=1200, bbox_inches='tight', pad_inches=0.2)  # Added padding

# Display the plot
plt.show()